# Example Notebook to load data

### Library imports

In [ ]:
import sys # os stands for "operating system" and includes read/write routines etc. 
import mkl
from src import utils # this is our own library of functions
mkl.set_num_threads(10)
sys.path.insert(0,r'C:\Users\WinDesktopMP\Documents\suite2p')
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
exp_db = []
exp_db = utils.add_exp(exp_db, 'TX80', '2022_06_17', '1')
iexp       = 0
timeline   = utils.load_exp_info(exp_db[iexp])
spks, xpos, ypos, iplane, stat, ops = utils.load_neurons(exp_db[iexp], dual_plane=True, baseline=True)

In [ ]:
neurons_atframes, subset_stim = utils.get_neurons_atframes(timeline,spks)
avg_response, csig = utils.get_tuned_neurons(neurons_atframes, subset_stim)
csig.mean()

Gets the 8x4 stimuli:

In [ ]:
cats, stim_idx = utils.get_stim_class_and_samples_ix(subset_stim, n_categories=8, samples_per_cat=4)